
First make sure to run your ssh tunnel with:  
GCP: 
  `gcloud compute ssh --zone us-east1-b --project katana-internal31 katana-controller -- -NL 8080:127.0.0.1:8080 -vvv`
<br> <br>
AWS: 
  `ssh -i ~/.ssh/id_rsa -N -L 8080:54.162.203.170:8080 ubuntu@54.162.203.170 -vvv` 
<br><br>  

In [1]:
######## setup variables ##########
gs_prefix="gs://anand-31-simple-files/customer-claims/customer-360"
cluster_name="katana-internal31"
partitions = 1
platform_prefix=gs_prefix

In [2]:
import os                                                                        
import time                                                                      
import json
import pandas as pd
from timeit import default_timer as timer
from katana.remote import aio, analytics, import_data, trace

from katana import remote
from katana.remote import import_data


os.environ["KATANA_SERVER_ADDRESS"] = "host.docker.internal:8080"

pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

In [3]:
rc=remote.Client()
rc.server_version

<coroutine object Client.server_version at 0x7ff669fc3740>

In [4]:

graph = rc.create_graph(
    num_partitions=partitions,
    name="customer-claims"
)

In [5]:
graph.graph_id

'Gu21wDbAL7pDCHmsKiXd4qZMbzvSfTvifDXZxkpyNQJq'

In [6]:
test_node_file = platform_prefix + "/42_NodeHeader.txt"
test_edge_file = platform_prefix + "/43_EdgeHeader.txt"
input_path=platform_prefix 


In [9]:
start = timer()
print("Importing graph from CSV files...")
import_data.csv(
    graph,
    input_node_path=test_node_file,
    input_edge_path=test_edge_file,
    input_dir=input_path,
#    have_headers=True,
    data_delimiter="|",
    schema_delimiter="|"
    #partition_policy="random-oec"
)
end = timer()
print(f"  import: {end - start:.1f} seconds") 

Importing graph from CSV files...


          0/? [?op/s]

  import: 26.9 seconds


In [34]:
%%time
graph.num_nodes()

          0/? [?op/s]

CPU times: user 117 ms, sys: 60.5 ms, total: 178 ms
Wall time: 7.93 s


157927

<b> Import ends here: DATA LOADING COMPLETE </b>

In [33]:
%%time
#check transaction objects
query1="""
        MATCH (c:Agent)-[:IS_LOCATED_IN]-(l:Location)

        RETURN c,l LIMIT 50"""
result = graph.query(query1, contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 184 ms, sys: 41.1 ms, total: 225 ms
Wall time: 3.72 s
